In [2]:
import math
import numpy as np
import tensorflow as tf
from __future__ import division, print_function, unicode_literals
from tensorflow.examples.tutorials.mnist import input_data as mnist_data

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

Below, we create a CNN network for the MNIST dataset with the following architecture:
* **Conv. layer 1:** computes 32 feature maps using a 5x5 filter with ReLU activation.
* **Pooling layer 1:** max pooling layer with a 2x2 filter and stride of 2.
* **Conv. layer 2:** computes 64 feature maps using a 5x5 filter.
* **Pooling layer 2:** max pooling layer with a 2x2 filter and stride of 2.
* **Dense layer:** densely connected layer with 1024 neurons.
* **Logits layer**

First we make the network manually.

In [3]:
########################################
# load the mnist data
########################################
mnist = mnist_data.read_data_sets("data", one_hot=True, reshape=False, validation_size=0)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [7]:
# manual building layers
reset_graph()

n_epochs = 1000
batch_size = 100


########################################
# define placeholders
########################################
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
y_true = tf.placeholder(tf.float32, [None, 10])
pkeep = tf.placeholder(tf.float32)

########################################
# build the model
########################################
# Convolutional Layer #1
# Computes 32 feature maps using a 5x5 filter with ReLU activation.
# Padding is added to preserve width and height.
# Input Tensor Shape: [batch_size, 28, 28, 1]
# Output Tensor Shape: [batch_size, 28, 28, 32]
W1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))  # 5x5 filter, 1 input channel, 32 output channels
B1 = tf.Variable(tf.constant(0.1, tf.float32, [32]))
stride = 1  # output is 28x28
conv1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding="SAME") + B1)

# Pooling Layer #1
# First max pooling layer with a 2x2 filter and stride of 2
# Input Tensor Shape: [batch_size, 28, 28, 32]
# Output Tensor Shape: [batch_size, 14, 14, 32]
pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

# Convolutional Layer #2
# Computes 64 features using a 5x5 filter.
# Padding is added to preserve width and height.
# Input Tensor Shape: [batch_size, 14, 14, 32]
# Output Tensor Shape: [batch_size, 14, 14, 64]
W2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))  # 5x5 filter, 32 input channel, 64 output channels
B2 = tf.Variable(tf.constant(0.1, tf.float32, [64]))
stride = 1  # output is 28x28
conv2 = tf.nn.relu(tf.nn.conv2d(pool1, W2, strides=[1, stride, stride, 1], padding="SAME") + B2)

# Pooling Layer #2
# Second max pooling layer with a 2x2 filter and stride of 2
# Input Tensor Shape: [batch_size, 14, 14, 64]
# Output Tensor Shape: [batch_size, 7, 7, 64]
pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

# Flatten tensor into a batch of vectors
# Input Tensor Shape: [batch_size, 7, 7, 64]
# Output Tensor Shape: [batch_size, 7 * 7 * 64]
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

# Dense Layer
# Densely connected layer with 1024 neurons
# Input Tensor Shape: [batch_size, 7 * 7 * 64]
# Output Tensor Shape: [batch_size, 1024]
W3 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1))  # 7x7x64 feature, 1024 neurons
B3 = tf.Variable(tf.constant(0.1, tf.float32, [1024]))
dense = tf.nn.relu(tf.matmul(pool2_flat, W3) + B3)

# Add dropout operation
dropout = tf.nn.dropout(dense, pkeep)

# Logits layer
# Input Tensor Shape: [batch_size, 1024]
# Output Tensor Shape: [batch_size, 10]
W4 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))  # 1024 feature, 10 neurons
B4 = tf.Variable(tf.constant(0.1, tf.float32, [10]))
logits = tf.matmul(dropout, W4) + B4
y_hat = tf.nn.softmax(logits)

########################################
# define the cost and accuracy functions
########################################
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_true)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

correct_prediction = tf.equal(tf.argmax(y_hat, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

########################################
# define the optimizer
########################################
lr = 0.003
optimizer = tf.train.AdamOptimizer(lr)
train_step = optimizer.minimize(cross_entropy)

########################################
# execute the model
########################################
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for i in range(n_epochs):
        # load batch of images and correct answers
        batch_X, batch_y = mnist.train.next_batch(batch_size)
        
        out_weights = W4.eval()
        print(np.sum(out_weights))
        # learning rate decay
        if i % 100 == 0:
            a, c = sess.run([accuracy, cross_entropy], feed_dict={X: batch_X, y_true: batch_y, pkeep: 1.0})
            print('epoch {}: accurecy = {}, loss = {}'.format(i, a, c))

        # train
        sess.run(train_step, feed_dict={X: batch_X, y_true: batch_y, pkeep: 0.75})
        
    a, c = sess.run([accuracy, cross_entropy], feed_dict={X: mnist.test.images, y_true: mnist.test.labels, pkeep: 1.0})
    print('test data: accurecy = {}, loss = {}'.format(a, c))  

-8.47429
epoch 0: accurecy = 0.11999999731779099, loss = 873.5614624023438
-3.2021
4.34828


KeyboardInterrupt: 

Now, we take advantage of the `tf.layers` to build the network.

In [4]:
reset_graph()

n_epochs = 1000
batch_size = 100

########################################
# load the mnist data
########################################
mnist = mnist_data.read_data_sets("data", one_hot=True, reshape=False, validation_size=0)

#######################################
# defineplaceholders
########################################
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
y_true = tf.placeholder(tf.float32, [None, 10])
pkeep = tf.placeholder(tf.float32)

########################################
# build the model
########################################
# Convolutional Layer #1
# Computes 32 feature maps using a 5x5 filter with ReLU activation.
# Padding is added to preserve width and height.
# Input Tensor Shape: [batch_size, 28, 28, 1]
# Output Tensor Shape: [batch_size, 28, 28, 32]
conv1 = tf.layers.conv2d(inputs=X, filters=32, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)

# Pooling Layer #1
# First max pooling layer with a 2x2 filter and stride of 2
# Input Tensor Shape: [batch_size, 28, 28, 32]
# Output Tensor Shape: [batch_size, 14, 14, 32]
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

# Convolutional Layer #2
# Computes 64 features using a 5x5 filter.
# Padding is added to preserve width and height.
# Input Tensor Shape: [batch_size, 14, 14, 32]
# Output Tensor Shape: [batch_size, 14, 14, 64]
conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)

# Pooling Layer #2
# Second max pooling layer with a 2x2 filter and stride of 2
# Input Tensor Shape: [batch_size, 14, 14, 64]
# Output Tensor Shape: [batch_size, 7, 7, 64]
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

# Flatten tensor into a batch of vectors
# Input Tensor Shape: [batch_size, 7, 7, 64]
# Output Tensor Shape: [batch_size, 7 * 7 * 64]
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

# Dense Layer
# Densely connected layer with 1024 neurons
# Input Tensor Shape: [batch_size, 7 * 7 * 64]
# Output Tensor Shape: [batch_size, 1024]
dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

# Add dropout operation
dropout = tf.layers.dropout(inputs=dense, rate=pkeep)

# Logits layer
# Input Tensor Shape: [batch_size, 1024]
# Output Tensor Shape: [batch_size, 10]
logits = tf.layers.dense(inputs=dropout, units=10)
y_hat = tf.nn.softmax(logits)

########################################
# define the cost and accuracy functions
########################################
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_true)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

correct_prediction = tf.equal(tf.argmax(y_hat, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

########################################
# define the optimizer
########################################
lr = 0.003
optimizer = tf.train.AdamOptimizer(lr)
train_step = optimizer.minimize(cross_entropy)

########################################
# execute the model
########################################
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for i in range(n_epochs):
        # load batch of images and correct answers
        batch_X, batch_y = mnist.train.next_batch(batch_size)
        
        # learning rate decay
        if i % 100 == 0:
            a, c = sess.run([accuracy, cross_entropy], feed_dict={X: batch_X, y_true: batch_y, pkeep: 1.0})
            print('epoch {}: accurecy = {}, loss = {}'.format(i, a, c))

        # train
        sess.run(train_step, feed_dict={X: batch_X, y_true: batch_y, pkeep: 0.75})
        
    a, c = sess.run([accuracy, cross_entropy], feed_dict={X: mnist.test.images, y_true: mnist.test.labels, pkeep: 1.0})
    print('test data: accurecy = {}, loss = {}'.format(a, c))  

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

epoch 0: accurecy = 0.05999999865889549, loss = 230.9563446044922


KeyboardInterrupt: 

In [ ]:
## Klesolution 
# · · · · · · · · · ·    (input data, 1-deep)                 X [batch, 28, 28, 1]
# @ @ @ @ @ @ @ @ @ @ -- conv. layer 6x6x1=>6 stride 1        W1 [5, 5, 1, 6]        B1 [6]
# ∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶                                         Y1_hat [batch, 28, 28, 6]
#   @ @ @ @ @ @ @ @   -- conv. layer 5x5x6=>12 stride 2       W2 [5, 5, 6, 12]        B2 [12]
#   ∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶                                           Y2_hat [batch, 14, 14, 12]
#     @ @ @ @ @ @     -- conv. layer 4x4x12=>24 stride 2      W3 [4, 4, 12, 24]       B3 [24]
#     ∶∶∶∶∶∶∶∶∶∶∶                                             Y3_hat [batch, 7, 7, 24] => reshaped to YY [batch, 7*7*24]
#      \x/x\x\x/ ✞    -- fully connected layer (relu+dropout) W4 [7*7*24, 200]       B4 [200]
#       · · · ·                                               Y4_hat [batch, 200]
#       \x/x\x/       -- fully connected layer (softmax)      W5 [200, 10]           B5 [10]
#        · · ·                                                Y_hat [batch, 10]


# to reset the Tensorflow default graph
reset_graph()

########################################
# define variables and placeholders
########################################
X = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
y_true = tf.placeholder(tf.float32, [None, 10])
pkeep = tf.placeholder(tf.float32)
learning_rate = tf.placeholder(tf.float32, shape=[])

# three convolutional layers with their channel counts, and a fully connected layer 
# (the last layer has 10 softmax neurons)
# the output depth of first three convolutional layers, are 6, 12, 24, and the size of fully connected
# layer is 200

# L1: conv. layer 6x6x1=>6 stride 1. 
stride = 1
conv1 = tf.layers.conv2d(
                         inputs = X, 
                         filters = 6, 
                         kernel_size = [6,6], 
                         strides = [stride, stride], 
                         padding = 'SAME',
                         activation = tf.nn.relu
                        )

# L2: conv. layer 5x5x6=>12 stride 2 
stride = 2
conv2 = tf.layers.conv2d(inputs = conv1,
                         filters = 12, 
                         kernel_size = [5,5], 
                         strides = [stride, stride], 
                         padding = 'SAME',
                         activation = tf.nn.relu
                        )
# L3: conv. layer 4x4x12=>24 stride 2
stride = 2
conv3 = tf.layers.conv2d(inputs = conv2,
                         filters = 24, 
                         kernel_size = [4,4], 
                         strides = [stride, stride], 
                         padding = 'SAME',
                         activation = tf.nn.relu
                        )

# note: if we want other initializers for kernel and bias we need to explicitly say so. 
# Let's try with standard and go from there (as results are not brilliant atm)
# L4: fully connected layer (relu+dropout) 200 neurons
dense = tf.layers.dense(inputs=conv3, 
                        units=200, 
                        use_bias = True, 
                        activation = tf.nn.relu
                       )

# Add dropout operation
dropout = tf.layers.dropout(inputs=dense, rate=pkeep)


# L5: fully connected layer (softmax) 10 neurons
logits = tf.layers.dense(inputs=dropout, 
                         units=10, 
                         use_bias = True
                        )

y_hat = tf.nn.softmax(logits)

########################################
# define the cost and accuracy functions
########################################
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_true)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

correct_prediction = tf.equal(tf.argmax(y_hat, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

########################################
# define the optimizer
########################################
lr = 0.003
optimizer = tf.train.AdamOptimizer(lr)
train_step = optimizer.minimize(cross_entropy)


########################################
# execute the model
########################################
init = tf.global_variables_initializer()
n_epochs = 5000
batch_size = 100
with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        # load batch of images and correct answers
        batch_X, batch_Y = mnist.train.next_batch(batch_size)
        
        if i % 100 == 0:
            print(batch_X.shape)
            a, c = sess.run([accuracy, cross_entropy], feed_dict={X: batch_X, y_true: batch_Y, pkeep: 1.0})
            print('epoch {}: accurecy = {}, loss = {}'.format(i, a, c))

        # train
        sess.run(train_step, feed_dict={X: batch_X, y_true: batch_Y, pkeep: 0.75})
        
    a, c = sess.run([accuracy, cross_entropy], feed_dict={X: mnist.test.images, y_true: mnist.test.labels, pkeep: 1.0})
    print('test data: accurecy = {}, loss = {}'.format(a, c))
    
    

Finally, we use Keras to build the above network.

In [5]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

reset_graph()

n_epochs = 1000
batch_size = 100

########################################
# load the mnist data
########################################
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Rscaling of the pixel values from 0 to 255 to the range between 0 and 1. It improves the learning speed.
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

########################################
# buid the model
########################################
model = Sequential()
model.add(Conv2D(32, kernel_size=[5, 5], activation='relu', input_shape=[28, 28, 1]))
model.add(MaxPooling2D(pool_size=[2, 2], strides=2))
model.add(Conv2D(64, kernel_size=[5, 5], activation='relu'))
model.add(MaxPooling2D(pool_size=[2, 2], strides=2))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, epochs=n_epochs, verbose=1, validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

NameError: name 'keras' is not defined

<img src="figs/19-letnet5.png" style="width: 600px;"/>
There are a few extra details to be noted:
* MNIST images are 28×28 pixels, but they are zero-padded to 32×32 pixels and normalized before being fed to the network. The rest of the network does not use any padding, which is why the size keeps shrinking as the image progresses through the network.
* The average pooling layers are slightly more complex than usual: each neuron computes the mean of its inputs, then multiplies the result by a learnable coefficient and adds a learnable bias term, then finally applies the activation function.
* Most neurons in layer C3 maps are connected to neurons in only three or four S2 maps (instead of all six S2 maps). See table 1 in the [paper](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf) for details.
* The output layer is a bit special: instead of computing the dot product of the inputs and the weight vector, each neuron outputs the square of the Euclidian distance between its input vector and its weight vector. Each output measures how much the image belongs to a particular digit class. The cross-entropy cost function is now preferred, as it penalizes bad predictions much more, producing larger gradients and thus converging faster.

# 11. Implement AlexNet
In the last section, you should implement **AlexNet** either using Tensorflow or Keras. Again, please take a look at its [paper](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) before start to implement it.
The AlexNet CNN architecture won the [ImageNet ILSVRC challenge](http://www.image-net.org/challenges/LSVRC/2012/) in 2012 by a large margin. It was developed by Alex Krizhevsky, Ilya Sutskever, and Geoffrey Hinton. It is quite similar to LeNet-5, only much larger and deeper, and it was the first to stack convolutional layers directly on top of each other, instead of stacking a pooling layer on top of each convolutional layer. The following table presents this architecture.
<img src="figs/20-alexnet.png" style="width: 600px;"/>
To train the model, we need a big dataset, however, in this assignment you are going to to assign the pretrained weights to your model, using `tf.Variable.assign`. You can download the pretrained weights from [bvlc_alexnet.npy](https://www.cs.toronto.edu/~guerzhoy/tf_alexnet/bvlc_alexnet.npy). This file is a NumPy array file created by the python. After you read this file, you will receive a python dictionary with a <key, value> pair for each layer. Each key is one of the layers names, e.g., `conv1`, and each value is a list of two values: (1) weights, and (2) biases of that layer. Part of the function to load the weights and biases to your model is given, and you need to complete it.
